In [25]:
from os.path import dirname, splitext
import networkx as nx
import sys
sys.path.append('/')
from itertools import chain
from functools import partial
from multiprocessing import Pool
from tqdm import tqdm
import matplotlib
# matplotlib.use('Agg')
from matplotlib import pyplot as plt
from sklearn.svm import SVR
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import KFold, train_test_split, ParameterGrid
from gklearn.utils.kernels import gaussiankernel, polynomialkernel
from gklearn.utils.graphdataset import get_dataset_attributes
from gklearn.utils.graph_files import load_dataset
from gklearn.utils.parallel import parallel_gm
from gklearn.utils import Dataset
from gklearn.utils.graphfiles import loadDataset
import numpy as np
from collections import Counter
from functools import partial
import time as tm

dslist = [{'name': 'Alkane', 'datafile': 'datasets/Alkane/dataset.ds','dataset_y': 'datasets/Alkane/dataset_boiling_point_names.txt', 'task':'regression'},
          {'name': 'Acyclic', 'datafile': 'datasets/acyclic/dataset_bps.ds','dataset_y': None,'task':'regression'},
          {'name': 'MAO', 'datafile': 'datasets/MAO/dataset.ds', 'dataset_y': None,'task':'classification'},
          {'name': 'PAH', 'datafile': 'datasets/PAH/dataset.ds', 'dataset_y': None,'task':'classification'}, # unlabeled
          {'name': 'MUTAG', 'datafile': 'datasets/MUTAG/MUTAG_A.txt', 'dataset_y': None,'task':'classification'}, # node/edge symb
           {'name': 'Letter-med', 'datafile': 'datasets/Letter-med/Letter-med_A.txt', 'dataset_y': None,'task':'classification'},
    # node nsymb
          {'name': 'ENZYMES', 'datafile': 'datasets/ENZYMES_txt/ENZYMES_A_sparse.txt', 'dataset_y': None,'task':'regression'}]
#           {'name': 'Fingerprint', 'datafile': 'datasets/Fingerprint/Fingerprint_A.txt','dataset_y': None}]

# To calculate the clustering coefficient
def calccluster(G):
    listcluster = []
    for node in G.nodes():
        neighbours=[n for n in nx.neighbors(G,node)]
        n_neighbors=len(neighbours)
        n_links=0
        if n_neighbors>1:
            for node1 in neighbours:
                for node2 in neighbours:
                    if G.has_edge(node1,node2) == True:
                        n_links=n_links+1
            n_links/=2.0 #because n_links is calculated twice
            clustering_coefficient=n_links/(0.5*n_neighbors*(n_neighbors-1))
            listcluster.append(clustering_coefficient)
        else:
            listcluster.append(0)
    return listcluster

# To calculate the objective function
def calcimp(G, node_label, weight):
    listimp = []
    nodelist = np.array(list(G.nodes())) # Necessary to scale node value to array index for easy addressing of label array and centrality array
    listcluster = calccluster(G)
    x = list(nx.betweenness_centrality(G, weight=weight).values())
    for node in G.nodes():
        neighbours=[n for n in nx.neighbors(G,node)]
        listlabel = list(nx.get_node_attributes(G, node_label).values())
#         print("node is ", node)
        cnt=0
        n_neighbours=len(neighbours)
        if n_neighbours == 0:
            n_neighbours = 1
        for node1 in neighbours:
            if listlabel[node-nodelist.min()] != listlabel[node1-nodelist.min()]:
                cnt=cnt+1
        listimp.append(-0.33*((cnt/n_neighbours)+x[node-nodelist.min()]+listcluster[node-nodelist.min()]))
    return listimp
        
    
def chooseDataset(ds_name):
    """Choose dataset according to name.
    """
    from gklearn.utils import Dataset
    
    dataset = Dataset()

    # no node labels (and no edge labels).
    if ds_name == 'Alkane':
        dataset.load_predefined_dataset(ds_name)
        dataset.trim_dataset(edge_required=False)
        irrelevant_labels = {'node_attrs': ['x', 'y', 'z'], 'edge_labels': ['bond_stereo']}
        dataset.remove_labels(**irrelevant_labels)
    # node symbolic labels.
    elif ds_name == 'Acyclic':
        dataset.load_predefined_dataset(ds_name)
        dataset.trim_dataset(edge_required=False)
        irrelevant_labels = {'node_attrs': ['x', 'y', 'z'], 'edge_labels': ['bond_stereo']}
        dataset.remove_labels(**irrelevant_labels)
    # node non-symbolic labels.
    elif ds_name == 'Letter-med':
        dataset.load_predefined_dataset(ds_name)
        dataset.trim_dataset(edge_required=False)
    # node symbolic and non-symbolic labels (and edge symbolic labels).
    elif ds_name == 'AIDS':
        dataset.load_predefined_dataset(ds_name)
        dataset.trim_dataset(edge_required=False)
    # edge non-symbolic labels (no node labels).
    elif ds_name == 'Fingerprint_edge':
        dataset.load_predefined_dataset('Fingerprint')
        dataset.trim_dataset(edge_required=True)
        irrelevant_labels = {'edge_attrs': ['orient', 'angle']}
        dataset.remove_labels(**irrelevant_labels)
    # edge non-symbolic labels (and node non-symbolic labels).
    elif ds_name == 'Fingerprint':
        dataset.load_predefined_dataset(ds_name)
        dataset.trim_dataset(edge_required=True)
    # edge symbolic and non-symbolic labels (and node symbolic and non-symbolic labels).
    elif ds_name == 'Cuneiform':
        dataset.load_predefined_dataset(ds_name)
        dataset.trim_dataset(edge_required=True)
        
    dataset.cut_graphs(range(0, 3))
    
    return dataset

def weisfeilerlehmankernel(*args, 
                           node_label='atom',
                           edge_weight=None,
                           edge_label='bond_type',
                           height=0,
                           base_kernel='subtree',
                           parallel=None,
                           n_jobs=None, 
                           verbose=True,
                           numlandmarks = 2,
                           sub_kernel=gaussiankernel):
    """Calculate Weisfeiler-Lehman kernels between graphs.
    
    Parameters
    ----------
    Gn : List of NetworkX graph
        List of graphs between which the kernels are calculated.
    
    G1, G2 : NetworkX graphs
        Two graphs between which the kernel is calculated.      
    node_label : string
        Node attribute used as label. The default node label is atom.       
    edge_label : string
        Edge attribute used as label. The default edge label is bond_type.      
    height : int
        Subtree height.
    base_kernel : string
        Base kernel used in each iteration of WL kernel. Only default 'subtree' 
        kernel can be applied for now.
    parallel : None
        Which paralleliztion method is applied to compute the kernel. No 
        parallelization can be applied for now.
    n_jobs : int
        Number of jobs for parallelization. The default is to use all 
        computational cores. This argument is only valid when one of the 
        parallelization method is applied and can be ignored for now.
    Return
    ------
    Kmatrix : Numpy matrix
        Kernel matrix, each element of which is the Weisfeiler-Lehman kernel between 2 praphs.
    Notes
    -----
    This function now supports WL subtree kernel only.
    """
#       The default base 
#       kernel is subtree kernel. For user-defined kernel, base_kernel is the 
#       name of the base kernel function used in each iteration of WL kernel. 
#       This function returns a Numpy matrix, each element of which is the 
#       user-defined Weisfeiler-Lehman kernel between 2 praphs.
#       pre-process

    base_kernel = base_kernel.lower()
    
    Gn = args[0] if len(args) == 1 else [args[0], args[1]] # arrange all graphs in a list
    Gn = [g.copy() for g in Gn]
    
    Kmatrix = np.zeros((len(Gn), len(Gn)))
    weight = None
    if edge_weight is None:
        if verbose:
            print('\n None edge weight specified. Set all weight to 1.\n')
    else:
        try:
            some_weight = list(nx.get_edge_attributes(Gn[0], edge_weight).values())[0]
            if isinstance(some_weight, (float, int)):
                weight = edge_weight
            else:
                if verbose:
                    print('\n Edge weight with name %s is not float or integer. Set all weight to 1.\n'% edge_weight)
        except:
            if verbose:
                print('\n Edge weight with name "%s" is not found in the edge attributes. Set all weight to 1.\n'% edge_weight)
    
    ds_attrs = get_dataset_attributes(Gn, attr_names=['node_labeled', 'node_attr_dim', 'edge_labeled', 'edge_attr_dim', 'is_directed'], 
                                      node_label=node_label, edge_label=edge_label)

    labeled = False
    if ds_attrs['node_labeled'] or ds_attrs['edge_labeled']:
        labeled = True
        if not ds_attrs['node_labeled']:
            for G in Gn:
                nx.set_node_attributes(G, '0', 'atom')
        if not ds_attrs['edge_labeled']:
            for G in Gn:
                nx.set_edge_attributes(G, '0', 'bond_type')

    start_time = tm.time()

    # for WL subtree kernel
    if base_kernel == 'subtree':           
        Gn, gn_landmarklist = _wl_kernel_do(Gn, node_label, weight, edge_label, height, parallel, n_jobs, verbose, numlandmarks)
    
    # get all canonical keys of all graphs before calculating kernels to save 
    # time, but this may cost a lot of memory for large dataset.
    canonkeys = []
    for g in (tqdm(Gn, desc='getting canonkeys', file=sys.stdout) if verbose else Gn):
        canonkeys.append(get_canonkeys(g, node_label, edge_label, labeled, 
                                       ds_attrs['is_directed']))

    # compute kernels.
    from itertools import combinations_with_replacement
    itr = combinations_with_replacement(range(0, len(Gn)), 2)
    for i, j in (tqdm(itr, desc='getting canonkeys', file=sys.stdout) if verbose else itr):
        Kmatrix[i][j] = _treeletkernel_do(canonkeys[i], canonkeys[j], sub_kernel)
        Kmatrix[j][i] = Kmatrix[i][j] # @todo: no directed graph considered?
        
    run_time = tm.time() - start_time
    if verbose:
        print("\n --- treelet kernel matrix of size %d built in %s seconds ---"% (len(Gn), run_time))

    return Kmatrix, run_time



#<<<<<<<<<<<<<< THIS CALCULATES THE WL LABELS AND RETURNS A SUBGRAPH >>>>>>>>>>>>>>>>>>>#
def _wl_kernel_do(Gn, node_label, weight, edge_label, height, parallel, n_jobs, verbose, numlandmarks):
    """Calculate Weisfeiler-Lehman kernels between graphs.
    Parameters
    ----------
    Gn : List of NetworkX graph
        List of graphs between which the kernels are calculated.       
    node_label : string
        node attribute used as label.
    edge_label : string
        edge attribute used as label.     
    height : int
        wl height.
    Return
    ------
    Kmatrix : Numpy matrix
        Kernel matrix, each element of which is the Weisfeiler-Lehman kernel between 2 praphs.
    """
    height = int(height)
    Kmatrix = np.zeros((len(Gn), len(Gn)))
    
    # initial for height = 0
    all_num_of_each_label = [] # number of occurence of each label in each graph in this iteration
    
    all_labels_ori = [] # Initial lsit of all labels
    
    # for each graph
    for G in Gn:
        # get the set of original labels
        labels_ori = list(nx.get_node_attributes(G, node_label).values())
        # number of occurence of each label in G
        all_num_of_each_label.append(dict(Counter(labels_ori)))
        all_labels_ori.append(labels_ori)
#         print("Clustering coefficients are:", type(nx.clustering(G)))
#     print("Original labels are: \n", all_labels_ori)
#     print("All numbers of each labels are: \n", all_num_of_each_label)
    # calculate subtree kernel with the 0th iteration and add it to the final kernel
#     compute_kernel_matrix(Kmatrix, all_num_of_each_label, Gn, parallel, n_jobs, False)

    # iterate each height
    for h in range(1, height + 1):
        print("<<<<<<<<< ITERATION", h, ">>>>>>>>>>>")
        all_set_compressed = {} # a dictionary mapping original labels to new ones in all graphs in this iteration
        num_of_labels_occured = 0 # number of the set of letters that occur before as node labels at least once in all graphs
#       all_labels_ori = set() # all unique orignal labels in all graphs in this iteration
        all_num_of_each_label = [] # number of occurence of each label in G


        for idx, G in enumerate(Gn):

            all_multisets = []
            for node, attrs in G.nodes(data=True):
                # Multiset-label determination.
                multiset = [G.nodes[neighbors][node_label] for neighbors in G[node]]
                # sorting each multiset
                multiset.sort()
                multiset = [attrs[node_label]] + multiset # add the prefix 
                all_multisets.append(tuple(multiset))
#             print("All multisets are:", all_multisets)
            # label compression
            set_unique = list(set(all_multisets)) # set of unique multiset labels
            # a dictionary mapping original labels to new ones. 
            set_compressed = {}
            # if a label occured before, assign its former compressed label, 
            # else assign the number of labels occured + 1 as the compressed label. 
            for value in set_unique:
                if value in all_set_compressed.keys():
                    set_compressed.update({value: all_set_compressed[value]})
                else:
                    set_compressed.update({value: str(num_of_labels_occured + 1)})
                    num_of_labels_occured += 1

            all_set_compressed.update(set_compressed)

            # relabel nodes
            for idx, node in enumerate(G.nodes()):
                G.nodes[node][node_label] = set_compressed[all_multisets[idx]]
            
#             print("Changed Nodes are", list(nx.get_node_attributes(G,node_label).values()))
#             print("New clustering coefficient is", nx.clustering(G))
            # get the set of compressed labels
            labels_comp = list(nx.get_node_attributes(G, node_label).values())
#             print("compressed labels are: ", labels_comp)
#           all_labels_ori.update(labels_comp)
            all_num_of_each_label.append(dict(Counter(labels_comp)))
    
    # Compute the value for every node of every graph
    listcomp = [] # list of computed values, has the vertices instead
    listval = [] # list of lists having actual values
    for G in Gn:
        x = calcimp(G, node_label, weight)
        listval.append(x)
        y = np.array(x)
        z = np.argsort(y)
        listcomp.append(z)
    # Now select those vertices having max value of function. Remember, include atleast one vertex of each label
#     print("SORTED NODES ARE HERE YOOOOOOOOOOOOO:\n", listcomp)
#     print("SORTED VALUES ARE HERE YOOOOOOOOOOOO:\n",listval)
    print("<<<<<<<<<<<<<<<<< TIME TO FIND LANDMARKS YOOOOO >>>>>>>>>>>>>>>>")
    gn_landmarklist = [] # List of landmarks of all graphs
    for i in range(len(listcomp)):
        size = len(all_labels_ori[i])
        num_landmarks = numlandmarks if numlandmarks < size else size
        set_uniq = set(all_labels_ori[i])
        list_uniq = list(set_uniq)
        uniqsize = len(list_uniq)
        uniqcount = np.zeros(uniqsize)
        gi_landmarklist = []
        currlandmarks = 0 # Number of landmarks extracted
        for j in range(len(listcomp[i])):
            for k in range(uniqsize):
                if currlandmarks < num_landmarks:
                    if ((all_labels_ori[i][listcomp[i][j]] == list_uniq[k]) and (uniqcount[k] == 0)):
                        uniqcount[k] = uniqcount[k] + 1
                        gi_landmarklist.append(listcomp[i][j])
                        currlandmarks=currlandmarks+1
                else:
                    break
                        
        if currlandmarks < num_landmarks:
            for x in range(len(listcomp[i])):
                if currlandmarks < num_landmarks:
                    if (listcomp[i][x] in gi_landmarklist):
                        continue
                    else:
                        gi_landmarklist.append(listcomp[i][x])
                        currlandmarks=currlandmarks+1
                else:
                    break
                    
        gn_landmarklist.append(gi_landmarklist)     
#     print("<<<<<<<<<<<<<<<<<< FINAL LIST OF LANDMARKS ARE HERE >>>>>>>>>>>>>>>>>\n", gn_landmarklist)
    
    return extract_subgraphs(Gn, gn_landmarklist), gn_landmarklist
        
    

def extract_subgraphs(Gn, gn_landmarklist):
    Gn_new = []
    for i in range(len(Gn)):
        G1 = Gn[i].subgraph(gn_landmarklist[i]).copy()
        Gn_new.append(G1)
    return Gn_new
    
##################### CODE AS IT IS BEGINS ################################################################################

def _treeletkernel_do(canonkey1, canonkey2, sub_kernel):
    """Calculate treelet graph kernel between 2 graphs.
    
    Parameters
    ----------
    canonkey1, canonkey2 : list
        List of canonical keys in 2 graphs, where each key is represented by a string.
        
    Return
    ------
    kernel : float
        Treelet Kernel between 2 graphs.
    """
    keys = set(canonkey1.keys()) & set(canonkey2.keys()) # find same canonical keys in both graphs
    vector1 = np.array([(canonkey1[key] if (key in canonkey1.keys()) else 0) for key in keys])
    vector2 = np.array([(canonkey2[key] if (key in canonkey2.keys()) else 0) for key in keys]) 
    kernel = sub_kernel(vector1, vector2) 
    return kernel


def wrapper_treeletkernel_do(sub_kernel, itr):
    i = itr[0]
    j = itr[1]
    return i, j, _treeletkernel_do(G_canonkeys[i], G_canonkeys[j], sub_kernel)


def get_canonkeys(G, node_label, edge_label, labeled, is_directed):
    """Generate canonical keys of all treelets in a graph.
    
    Parameters
    ----------
    G : NetworkX graphs
        The graph in which keys are generated.
    node_label : string
        node attribute used as label. The default node label is atom.       
    edge_label : string
        edge attribute used as label. The default edge label is bond_type.
    labeled : boolean
        Whether the graphs are labeled. The default is True.
        
    Return
    ------
    canonkey/canonkey_l : dict
        For unlabeled graphs, canonkey is a dictionary which records amount of 
        every tree pattern. For labeled graphs, canonkey_l is one which keeps 
        track of amount of every treelet.
    """
    patterns = {} # a dictionary which consists of lists of patterns for all graphlet.
    canonkey = {} # canonical key, a dictionary which records amount of every tree pattern.

    ### structural analysis ###
    ### In this section, a list of patterns is generated for each graphlet, 
    ### where every pattern is represented by nodes ordered by Morgan's 
    ### extended labeling.
    # linear patterns
    patterns['0'] = G.nodes()
    canonkey['0'] = nx.number_of_nodes(G)
    for i in range(1, 6): # for i in range(1, 6):
        patterns[str(i)] = find_all_paths(G, i, is_directed)
        canonkey[str(i)] = len(patterns[str(i)])

    # n-star patterns
    patterns['3star'] = [[node] + [neighbor for neighbor in G[node]] for node in G.nodes() if G.degree(node) == 3]
    patterns['4star'] = [[node] + [neighbor for neighbor in G[node]] for node in G.nodes() if G.degree(node) == 4]
    patterns['5star'] = [[node] + [neighbor for neighbor in G[node]] for node in G.nodes() if G.degree(node) == 5]      
    # n-star patterns
    canonkey['6'] = len(patterns['3star'])
    canonkey['8'] = len(patterns['4star'])
    canonkey['d'] = len(patterns['5star'])

    # pattern 7
    patterns['7'] = [] # the 1st line of Table 1 in Ref [1]
    for pattern in patterns['3star']:
        for i in range(1, len(pattern)): # for each neighbor of node 0
            if G.degree(pattern[i]) >= 2:
                pattern_t = pattern[:]
                # set the node with degree >= 2 as the 4th node
                pattern_t[i], pattern_t[3] = pattern_t[3], pattern_t[i]
                for neighborx in G[pattern[i]]:
                    if neighborx != pattern[0]:
                        new_pattern = pattern_t + [neighborx]
                        patterns['7'].append(new_pattern)
    canonkey['7'] = len(patterns['7'])

    # pattern 11
    patterns['11'] = [] # the 4th line of Table 1 in Ref [1]
    for pattern in patterns['4star']:
        for i in range(1, len(pattern)):
            if G.degree(pattern[i]) >= 2:
                pattern_t = pattern[:]
                pattern_t[i], pattern_t[4] = pattern_t[4], pattern_t[i]
                for neighborx in G[pattern[i]]:
                    if neighborx != pattern[0]:
                        new_pattern = pattern_t + [ neighborx ]
                        patterns['11'].append(new_pattern)
    canonkey['b'] = len(patterns['11'])

    # pattern 12
    patterns['12'] = [] # the 5th line of Table 1 in Ref [1]
    rootlist = [] # a list of root nodes, whose extended labels are 3
    for pattern in patterns['3star']:
        if pattern[0] not in rootlist: # prevent to count the same pattern twice from each of the two root nodes
            rootlist.append(pattern[0])
            for i in range(1, len(pattern)):
                if G.degree(pattern[i]) >= 3:
                    rootlist.append(pattern[i])
                    pattern_t = pattern[:]
                    pattern_t[i], pattern_t[3] = pattern_t[3], pattern_t[i]
                    for neighborx1 in G[pattern[i]]:
                        if neighborx1 != pattern[0]:
                            for neighborx2 in G[pattern[i]]:
                                if neighborx1 > neighborx2 and neighborx2 != pattern[0]:
                                    new_pattern = pattern_t + [neighborx1] + [neighborx2]
#                        new_patterns = [ pattern + [neighborx1] + [neighborx2] for neighborx1 in G[pattern[i]] if neighborx1 != pattern[0] for neighborx2 in G[pattern[i]] if (neighborx1 > neighborx2 and neighborx2 != pattern[0]) ]
                                    patterns['12'].append(new_pattern)
    canonkey['c'] = int(len(patterns['12']) / 2)

    # pattern 9
    patterns['9'] = [] # the 2nd line of Table 1 in Ref [1]
    for pattern in patterns['3star']:
        for pairs in [ [neighbor1, neighbor2] for neighbor1 in G[pattern[0]] if G.degree(neighbor1) >= 2 \
            for neighbor2 in G[pattern[0]] if G.degree(neighbor2) >= 2 if neighbor1 > neighbor2 ]:
            pattern_t = pattern[:]
            # move nodes with extended labels 4 to specific position to correspond to their children
            pattern_t[pattern_t.index(pairs[0])], pattern_t[2] = pattern_t[2], pattern_t[pattern_t.index(pairs[0])]
            pattern_t[pattern_t.index(pairs[1])], pattern_t[3] = pattern_t[3], pattern_t[pattern_t.index(pairs[1])]
            for neighborx1 in G[pairs[0]]:
                if neighborx1 != pattern[0]:
                    for neighborx2 in G[pairs[1]]:
                        if neighborx2 != pattern[0]:
                            new_pattern = pattern_t + [neighborx1] + [neighborx2]
                            patterns['9'].append(new_pattern)
    canonkey['9'] = len(patterns['9'])

    # pattern 10
    patterns['10'] = [] # the 3rd line of Table 1 in Ref [1]
    for pattern in patterns['3star']:       
        for i in range(1, len(pattern)):
            if G.degree(pattern[i]) >= 2:
                for neighborx in G[pattern[i]]:
                    if neighborx != pattern[0] and G.degree(neighborx) >= 2:
                        pattern_t = pattern[:]
                        pattern_t[i], pattern_t[3] = pattern_t[3], pattern_t[i]
                        new_patterns = [ pattern_t + [neighborx] + [neighborxx] for neighborxx in G[neighborx] if neighborxx != pattern[i] ]
                        patterns['10'].extend(new_patterns)
    canonkey['a'] = len(patterns['10'])

    ### labeling information ###
    ### In this section, a list of canonical keys is generated for every 
    ### pattern obtained in the structural analysis section above, which is a 
    ### string corresponding to a unique treelet. A dictionary is built to keep
    ### track of the amount of every treelet.
    if labeled == True:
        canonkey_l = {} # canonical key, a dictionary which keeps track of amount of every treelet.

        # linear patterns
        canonkey_t = Counter(list(nx.get_node_attributes(G, node_label).values()))
        for key in canonkey_t:
            canonkey_l[('0', key)] = canonkey_t[key]

        for i in range(1, 6): # for i in range(1, 6):
            treelet = []
            for pattern in patterns[str(i)]:
                canonlist = list(chain.from_iterable((G.nodes[node][node_label], \
                    G[node][pattern[idx+1]][edge_label]) for idx, node in enumerate(pattern[:-1])))
                canonlist.append(G.nodes[pattern[-1]][node_label])
                canonkey_t = canonlist if canonlist < canonlist[::-1] else canonlist[::-1]
                treelet.append(tuple([str(i)] + canonkey_t))
            canonkey_l.update(Counter(treelet))

        # n-star patterns
        for i in range(3, 6):
            treelet = []
            for pattern in patterns[str(i) + 'star']:
                canonlist = [tuple((G.nodes[leaf][node_label], 
                                    G[leaf][pattern[0]][edge_label])) for leaf in pattern[1:]]
                canonlist.sort()
                canonlist = list(chain.from_iterable(canonlist))
                canonkey_t = tuple(['d' if i == 5 else str(i * 2)] + [G.nodes[pattern[0]][node_label]] + canonlist)
                treelet.append(canonkey_t)
            canonkey_l.update(Counter(treelet))

        # pattern 7
        treelet = []
        for pattern in patterns['7']:
            canonlist = [tuple((G.nodes[leaf][node_label], 
                                G[leaf][pattern[0]][edge_label])) for leaf in pattern[1:3]]
            canonlist.sort()
            canonlist = list(chain.from_iterable(canonlist))
            canonkey_t = tuple(['7'] + [G.nodes[pattern[0]][node_label]] + canonlist 
                               + [G.nodes[pattern[3]][node_label]] 
                               + [G[pattern[3]][pattern[0]][edge_label]]
                               + [G.nodes[pattern[4]][node_label]] 
                               + [G[pattern[4]][pattern[3]][edge_label]])
            treelet.append(canonkey_t)
        canonkey_l.update(Counter(treelet))

        # pattern 11
        treelet = []
        for pattern in patterns['11']:
            canonlist = [tuple((G.nodes[leaf][node_label], 
                                G[leaf][pattern[0]][edge_label])) for leaf in pattern[1:4]]
            canonlist.sort()
            canonlist = list(chain.from_iterable(canonlist))
            canonkey_t = tuple(['b'] + [G.nodes[pattern[0]][node_label]] + canonlist 
                               + [G.nodes[pattern[4]][node_label]] 
                               + [G[pattern[4]][pattern[0]][edge_label]]
                               + [G.nodes[pattern[5]][node_label]] 
                               + [G[pattern[5]][pattern[4]][edge_label]])
            treelet.append(canonkey_t)
        canonkey_l.update(Counter(treelet))

        # pattern 10
        treelet = []
        for pattern in patterns['10']:
            canonkey4 = [G.nodes[pattern[5]][node_label], G[pattern[5]][pattern[4]][edge_label]]
            canonlist = [tuple((G.nodes[leaf][node_label], 
                                G[leaf][pattern[0]][edge_label])) for leaf in pattern[1:3]]
            canonlist.sort()
            canonkey0 = list(chain.from_iterable(canonlist))
            canonkey_t = tuple(['a'] + [G.nodes[pattern[3]][node_label]] 
                               + [G.nodes[pattern[4]][node_label]] 
                               + [G[pattern[4]][pattern[3]][edge_label]] 
                               + [G.nodes[pattern[0]][node_label]] 
                               + [G[pattern[0]][pattern[3]][edge_label]] 
                               + canonkey4 + canonkey0)
            treelet.append(canonkey_t)
        canonkey_l.update(Counter(treelet))

        # pattern 12
        treelet = []
        for pattern in patterns['12']:
            canonlist0 = [tuple((G.nodes[leaf][node_label], 
                                 G[leaf][pattern[0]][edge_label])) for leaf in pattern[1:3]]
            canonlist0.sort()
            canonlist0 = list(chain.from_iterable(canonlist0))
            canonlist3 = [tuple((G.nodes[leaf][node_label], 
                                 G[leaf][pattern[3]][edge_label])) for leaf in pattern[4:6]]
            canonlist3.sort()
            canonlist3 = list(chain.from_iterable(canonlist3))
            
            # 2 possible key can be generated from 2 nodes with extended label 3, 
            # select the one with lower lexicographic order.
            canonkey_t1 = tuple(['c'] + [G.nodes[pattern[0]][node_label]] + canonlist0 
                                + [G.nodes[pattern[3]][node_label]] 
                                + [G[pattern[3]][pattern[0]][edge_label]] 
                                + canonlist3)
            canonkey_t2 = tuple(['c'] + [G.nodes[pattern[3]][node_label]] + canonlist3 
                                + [G.nodes[pattern[0]][node_label]] 
                                + [G[pattern[0]][pattern[3]][edge_label]] 
                                + canonlist0)
            treelet.append(canonkey_t1 if canonkey_t1 < canonkey_t2 else canonkey_t2)
        canonkey_l.update(Counter(treelet))

        # pattern 9
        treelet = []
        for pattern in patterns['9']:
            canonkey2 = [G.nodes[pattern[4]][node_label], G[pattern[4]][pattern[2]][edge_label]]
            canonkey3 = [G.nodes[pattern[5]][node_label], G[pattern[5]][pattern[3]][edge_label]]
            prekey2 = [G.nodes[pattern[2]][node_label], G[pattern[2]][pattern[0]][edge_label]]
            prekey3 = [G.nodes[pattern[3]][node_label], G[pattern[3]][pattern[0]][edge_label]]
            if prekey2 + canonkey2 < prekey3 + canonkey3:
                canonkey_t = [G.nodes[pattern[1]][node_label]] \
                             + [G[pattern[1]][pattern[0]][edge_label]] \
                             + prekey2 + prekey3 + canonkey2 + canonkey3
            else:
                canonkey_t = [G.nodes[pattern[1]][node_label]] \
                             + [G[pattern[1]][pattern[0]][edge_label]] \
                             + prekey3 + prekey2 + canonkey3 + canonkey2
            treelet.append(tuple(['9'] + [G.nodes[pattern[0]][node_label]] + canonkey_t))
        canonkey_l.update(Counter(treelet))

        return canonkey_l

    return canonkey


def wrapper_get_canonkeys(node_label, edge_label, labeled, is_directed, itr_item):
    g = itr_item[0]
    i = itr_item[1]
    return i, get_canonkeys(g, node_label, edge_label, labeled, is_directed)
    

def find_paths(G, source_node, length):
    """Find all paths with a certain length those start from a source node. 
    A recursive depth first search is applied.
    
    Parameters
    ----------
    G : NetworkX graphs
        The graph in which paths are searched.
    source_node : integer
        The number of the node from where all paths start.
    length : integer
        The length of paths.
        
    Return
    ------
    path : list of list
        List of paths retrieved, where each path is represented by a list of nodes.
    """
    if length == 0:
        return [[source_node]]
    path = [[source_node] + path for neighbor in G[source_node] \
        for path in find_paths(G, neighbor, length - 1) if source_node not in path]
    return path


def find_all_paths(G, length, is_directed):
    """Find all paths with a certain length in a graph. A recursive depth first
    search is applied.
    
    Parameters
    ----------
    G : NetworkX graphs
        The graph in which paths are searched.
    length : integer
        The length of paths.
        
    Return
    ------
    path : list of list
        List of paths retrieved, where each path is represented by a list of nodes.
    """
    all_paths = []
    for node in G:
        all_paths.extend(find_paths(G, node, length))
        
    if not is_directed:
        # For each path, two presentations are retrieved from its two extremities. 
        # Remove one of them.
        all_paths_r = [path[::-1] for path in all_paths]  
        for idx, path in enumerate(all_paths[:-1]):
            for path2 in all_paths_r[idx+1::]:
                if path == path2:
                    all_paths[idx] = []
                    break
        all_paths = list(filter(lambda a: a != [], all_paths))
            
    return all_paths

##################################CODE AS IT IS ENDS HERE##################################################################

#   compute_kernel_matrix(Kmatrix, all_num_of_each_label, Gn, parallel, n_jobs, False)

#   return Kmatrix

##################################CODE FOR TESTING BEGINS HERE#############################################################

# dataset, y_all = loadDataset(dslist[z]['datafile'], filename_y=dslist[z]['dataset_y'], extra_params=None)
# nx.draw_networkx(dataset[20])
# print("##########DEBUG##########")
# print(np.array(list(dataset[20].nodes())).min())
for z in range(len(dslist)):
    dataset, y_all = loadDataset(dslist[z]['datafile'], filename_y=dslist[z]['dataset_y'], extra_params=None)

    y_mid = np.array(y_all)

    y_mid = (y_mid - y_mid.min())/(y_mid.max()-y_mid.min())
    y_all = list(y_mid)
#     print(y_all)
#     nx.draw_networkx(dataset[0])

    # print(dataset[50].nodes)

    Kmatrix, time = weisfeilerlehmankernel(dataset[:], height = 10, numlandmarks =4)

    # estimator = treeletkernel
    param_grid_precomputed = {'sub_kernel': [gaussiankernel, polynomialkernel]}
    param_grid = [{'C': np.logspace(-10, 10, num=41, base=10)},
                  {'alpha': np.logspace(-10, 10, num=41, base=10)}]

    param_list_precomputed = list(ParameterGrid(param_grid_precomputed))
    param_list = list(ParameterGrid(param_grid))

    Kmatrix_diag = Kmatrix.diagonal().copy()

    # Removing 0 Diagonal elements
#     nb_g_ignore = 0
#     for idxk, diag in enumerate(Kmatrix_diag):
#         if diag == 0:
#             Kmatrix = np.delete(Kmatrix, (idxk - nb_g_ignore), axis=0)
#             Kmatrix = np.delete(Kmatrix, (idxk - nb_g_ignore), axis=1)
#             nb_g_ignore += 1

    # Normalise the Kmatrix
    for i in range (len(Kmatrix)):
        for j in range(i, len(Kmatrix)):
            if (Kmatrix_diag[i] == 0 or Kmatrix_diag[j] == 0):
                Kmatrix[i][j] = 0
            else:
                Kmatrix[i][j] /= np.sqrt(Kmatrix_diag[i]*Kmatrix_diag[j])

    gram_matrices = []
    gram_matrices.append(Kmatrix)
    y = y_all[:]

    indices = range(len(y))
    gm_now = gram_matrices[0].copy()

    X_app, X_test, y_app, y_test, idx_app, idx_test = train_test_split(gm_now, y, indices, test_size=0.1, random_state=1, shuffle=True)

    X_app = X_app[:, idx_app]
    X_test = X_test[:, idx_app]
    y_app = np.array(y_app)
    y_test = np.array(y_test)
    
    inner_cv = KFold(n_splits=10, shuffle=True, random_state=1)

    current_train_perf = []
    current_valid_perf = []
    current_test_perf = [] 
    
    if dslist[z]['task'] == 'regression':
        print("regression being done:\n")
        kr = SVR(kernel='precomputed')
        for train_index, valid_index in inner_cv.split(X_app):
            kr.fit(X_app[train_index, :][:, train_index],
                   y_app[train_index])

            # predict on the train, validation and test set
            y_pred_train = kr.predict(
                X_app[train_index, :][:, train_index])
            y_pred_valid = kr.predict(
                X_app[valid_index, :][:, train_index])
        #                    if trial == 0:     
        #                        print('y_pred_valid: ', y_pred_valid)
        #                        print()
            y_pred_test = kr.predict(
                X_test[:, train_index])

            # root mean squared errors
        # #     print(
        #              mean_squared_error(
        #                 y_app[train_index], y_pred_train))
            current_train_perf.append(
                np.sqrt(
                    mean_squared_error(
                        y_app[train_index], y_pred_train)))
            current_valid_perf.append(
                np.sqrt(
                    mean_squared_error(
                        y_app[valid_index], y_pred_valid)))
        #                    if trial == 0:
        #                        print(mean_squared_error(
        #                                y_app[valid_index], y_pred_valid))
            current_test_perf.append(
                np.sqrt(
                    mean_squared_error(
                        y_test, y_pred_test)))

#         print(y_test)
#         print(y_pred_test)
        train_pref = np.mean(
                        current_train_perf)
        val_pref = np.mean(
                        current_valid_perf)
        test_pref = np.mean(
                        current_test_perf)

        print(train_pref, val_pref, test_pref)
    else:
        print("classification being done\n")
        svc = SVC(kernel = 'precomputed')
        for train_index, valid_index in inner_cv.split(X_app):
            svc.fit(X_app[train_index, :][:, train_index],
                   y_app[train_index])

            # predict on the train, validation and test set
            y_pred_train = svc.predict(
                X_app[train_index, :][:, train_index])
            y_pred_valid = svc.predict(
                X_app[valid_index, :][:, train_index])
            y_pred_test = svc.predict(
                X_test[:, train_index])

            # root mean squared errors
            current_train_perf.append(
                accuracy_score(y_app[train_index],
                               y_pred_train))
            current_valid_perf.append(
                accuracy_score(y_app[valid_index],
                               y_pred_valid))
            current_test_perf.append(
                accuracy_score(y_test, y_pred_test))
            
        train_pref = np.mean(
            current_train_perf)
        val_pref = np.mean(
            current_valid_perf)
        test_pref = np.mean(
            current_test_perf)
        print(train_pref, val_pref, test_pref)
            
            
        

ValueError: not enough values to unpack (expected 3, got 2)

In [16]:
import numpy as np

a = np.array([[50,5],[2,6],[9,5.5]])
print(np.argsort(a, axis = 0))

[[1 0]
 [2 2]
 [0 1]]
